In [1]:
!pip install -q -U torch transformers peft bitsandbytes accelerate datasets scipy google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.

In [2]:
from google.colab import auth
auth.authenticate_user()

print("Environment Ready & Authenticated!")

Environment Ready & Authenticated!


In [3]:
import os
from google.cloud import aiplatform
import vertexai

In [4]:
PROJECT_ID = "boston311-mlops"
BUCKET_NAME = "boston311-mlops-data"
REGION = "us-central1"
EXPERIMENT_NAME = "boston311-sql-experiment"
MODEL_DISPLAY_NAME = "boston311-sqlcoder-tuned"

In [31]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=f"gs://{BUCKET_NAME}")
aiplatform.start_run(EXPERIMENT_NAME)

In [6]:
print(f"Downloading training data...")
!gsutil cp gs://{BUCKET_NAME}/training_data.jsonl .

if os.path.exists("training_data.jsonl"):
    print("Data ready.")
else:
    raise FileNotFoundError("Training_data.jsonl not found. Check bucket name.")

Copying gs://boston311-mlops-data/training_data.jsonl...
/ [1 files][ 62.6 KiB/ 62.6 KiB]                                                
Operation completed over 1 objects/62.6 KiB.                                     
Data ready.


In [7]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from google.cloud import aiplatform

In [8]:
MODEL_ID = "defog/sqlcoder-7b-2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading base model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

In [9]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)
model = get_peft_model(model, peft_config)

In [10]:
def format_prompt(sample):
    return {"text": f"{sample['input']}{sample['completion']}{tokenizer.eos_token}"}

In [11]:
dataset = load_dataset("json", data_files="training_data.jsonl", split="train")
dataset = dataset.train_test_split(test_size=0.1)

train_ds = dataset["train"].map(format_prompt).map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=512), batched=True)
eval_ds = dataset["test"].map(format_prompt).map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=512), batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [15]:
print("Starting Training...")
# aiplatform.start_run("run-v1")

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=5,  # Kept low for speed; increase to 5 for better results
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="no",
    report_to="none"     # We will log manually to Vertex below if needed, or rely on console
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Starting Training...


In [16]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.826500
20,0.309900
30,0.204200


TrainOutput(global_step=35, training_loss=0.4070371866226196, metrics={'train_runtime': 398.5109, 'train_samples_per_second': 0.703, 'train_steps_per_second': 0.088, 'total_flos': 5697860146298880.0, 'train_loss': 0.4070371866226196, 'epoch': 5.0})

In [18]:
test_cases = [
    {
        "input": "### Task\nGenerate a BigQuery SQL query to answer [QUESTION]show me open cases on commonwealth ave involving trees[/QUESTION]\n\n### Schema\nCREATE TABLE `boston311.service_requests_2025` (case_enquiry_id INTEGER, open_dt TIMESTAMP, closed_dt TIMESTAMP, sla_target_dt TIMESTAMP, case_status STRING, on_time STRING, closure_reason STRING, source STRING, department STRING, subject STRING, reason STRING, type STRING, queue STRING, case_title STRING, neighborhood STRING, location STRING, location_street_name STRING, location_zipcode INTEGER, latitude FLOAT, longitude FLOAT, geom_4326 STRING, precinct STRING, ward STRING, city_council_district STRING, fire_district STRING, police_district STRING, pwd_district STRING, neighborhood_services_district STRING, submitted_photo STRING, closed_photo STRING);\n\n### Hints\ncase_status='Open'\nreason='Trees'\n\n### Answer\n",
        "completion": ""
    },
    {
        "input": "### Task\nGenerate a BigQuery SQL query to answer [QUESTION]which department has the highest percentage of overdue cases[/QUESTION]\n\n### Schema\nCREATE TABLE `boston311.service_requests_2025` (case_enquiry_id INTEGER, open_dt TIMESTAMP, closed_dt TIMESTAMP, sla_target_dt TIMESTAMP, case_status STRING, on_time STRING, closure_reason STRING, source STRING, department STRING, subject STRING, reason STRING, type STRING, queue STRING, case_title STRING, neighborhood STRING, location STRING, location_street_name STRING, location_zipcode INTEGER, latitude FLOAT, longitude FLOAT, geom_4326 STRING, precinct STRING, ward STRING, city_council_district STRING, fire_district STRING, police_district STRING, pwd_district STRING, neighborhood_services_district STRING, submitted_photo STRING, closed_photo STRING);\n\n### Hints\n\n\n### Answer\n",
        "completion": ""
    },
    {
        "input": "### Task\nGenerate a BigQuery SQL query to answer [QUESTION]list cases that have been open for more than 100 days[/QUESTION]\n\n### Schema\nCREATE TABLE `boston311.service_requests_2025` (case_enquiry_id INTEGER, open_dt TIMESTAMP, closed_dt TIMESTAMP, sla_target_dt TIMESTAMP, case_status STRING, on_time STRING, closure_reason STRING, source STRING, department STRING, subject STRING, reason STRING, type STRING, queue STRING, case_title STRING, neighborhood STRING, location STRING, location_street_name STRING, location_zipcode INTEGER, latitude FLOAT, longitude FLOAT, geom_4326 STRING, precinct STRING, ward STRING, city_council_district STRING, fire_district STRING, police_district STRING, pwd_district STRING, neighborhood_services_district STRING, submitted_photo STRING, closed_photo STRING);\n\n### Hints\ncase_status='Open'\n\n### Answer\n",
        "completion": ""
    },
    {
        "input": "### Task\nGenerate a BigQuery SQL query to answer [QUESTION]find requests where the source is not the app and location is missing[/QUESTION]\n\n### Schema\nCREATE TABLE `boston311.service_requests_2025` (case_enquiry_id INTEGER, open_dt TIMESTAMP, closed_dt TIMESTAMP, sla_target_dt TIMESTAMP, case_status STRING, on_time STRING, closure_reason STRING, source STRING, department STRING, subject STRING, reason STRING, type STRING, queue STRING, case_title STRING, neighborhood STRING, location STRING, location_street_name STRING, location_zipcode INTEGER, latitude FLOAT, longitude FLOAT, geom_4326 STRING, precinct STRING, ward STRING, city_council_district STRING, fire_district STRING, police_district STRING, pwd_district STRING, neighborhood_services_district STRING, submitted_photo STRING, closed_photo STRING);\n\n### Hints\nsource='Citizens Connect App'\n\n### Answer\n",
        "completion": ""
    },
    {
        "input": "### Task\nGenerate a BigQuery SQL query to answer [QUESTION]which hour of the day sees the most parking tickets[/QUESTION]\n\n### Schema\nCREATE TABLE `boston311.service_requests_2025` (case_enquiry_id INTEGER, open_dt TIMESTAMP, closed_dt TIMESTAMP, sla_target_dt TIMESTAMP, case_status STRING, on_time STRING, closure_reason STRING, source STRING, department STRING, subject STRING, reason STRING, type STRING, queue STRING, case_title STRING, neighborhood STRING, location STRING, location_street_name STRING, location_zipcode INTEGER, latitude FLOAT, longitude FLOAT, geom_4326 STRING, precinct STRING, ward STRING, city_council_district STRING, fire_district STRING, police_district STRING, pwd_district STRING, neighborhood_services_district STRING, submitted_photo STRING, closed_photo STRING);\n\n### Hints\ntype='Parking Enforcement'\n\n### Answer\n",
        "completion": ""
    },
    {
        "input": "### Task\nGenerate a BigQuery SQL query to answer [QUESTION]breakdown of rodent activity by neighborhood and month for 2025[/QUESTION]\n\n### Schema\nCREATE TABLE `boston311.service_requests_2025` (case_enquiry_id INTEGER, open_dt TIMESTAMP, closed_dt TIMESTAMP, sla_target_dt TIMESTAMP, case_status STRING, on_time STRING, closure_reason STRING, source STRING, department STRING, subject STRING, reason STRING, type STRING, queue STRING, case_title STRING, neighborhood STRING, location STRING, location_street_name STRING, location_zipcode INTEGER, latitude FLOAT, longitude FLOAT, geom_4326 STRING, precinct STRING, ward STRING, city_council_district STRING, fire_district STRING, police_district STRING, pwd_district STRING, neighborhood_services_district STRING, submitted_photo STRING, closed_photo STRING);\n\n### Hints\ntype='Rodent Activity'\n\n### Answer\n",
        "completion": ""
    }
]

In [19]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32016, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [28]:
def generate_sql(prompt_text):
    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "### Answer" in generated_text:
        return generated_text.split("### Answer")[1].strip()
    else:
        return generated_text

In [29]:
print("Running Inference on Test Cases...\n")

for i, test in enumerate(test_cases):
    print(f"--- Test Case {i+1} ---")

    question_part = test['input'].split("[QUESTION]")[1].split("[/QUESTION]")[0]
    print(f"Question: {question_part}")

    # Generate the SQL
    sql_output = generate_sql(test['input'])

    print(f"Generated SQL:\n{sql_output}\n")

Running Inference on Test Cases...

--- Test Case 1 ---
Question: show me open cases on commonwealth ave involving trees
Generated SQL:
SELECT * FROM `boston311.service_requests_2025` WHERE case_status = 'Open' AND reason = 'Trees' AND neighborhood = 'Commonwealth Ave';

--- Test Case 2 ---
Question: which department has the highest percentage of overdue cases
Generated SQL:
SELECT department, COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS percentage FROM `boston311.service_requests_2025` WHERE case_status = 'Closed' AND on_time = 'Late' AND closure_reason = 'Case Closed' GROUP BY department ORDER BY percentage DESC LIMIT 1;

--- Test Case 3 ---
Question: list cases that have been open for more than 100 days
Generated SQL:
SELECT * FROM `boston311.service_requests_2025` WHERE case_status = 'Open' AND EXTRACT(DAY FROM CURRENT_DATE - open_dt) > 100;

--- Test Case 4 ---
Question: find requests where the source is not the app and location is missing
Generated SQL:
SELECT * FROM `boston311.ser

In [32]:
aiplatform.log_params({"epochs": 2, "batch_size": 4, "lora_r": 16})
aiplatform.end_run()

print("Training Complete.")

Training Complete.


In [33]:
OUTPUT_DIR = "./sqlcoder_adapter"
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

('./sqlcoder_adapter/tokenizer_config.json',
 './sqlcoder_adapter/special_tokens_map.json',
 './sqlcoder_adapter/tokenizer.model',
 './sqlcoder_adapter/added_tokens.json',
 './sqlcoder_adapter/tokenizer.json')

In [36]:
MODEL_BUCKET_NAME = 'boston311-chatbot-model'

In [37]:
GCS_MODEL_PATH = f"gs://{MODEL_BUCKET_NAME}/models/{MODEL_DISPLAY_NAME}/"
print(f"Uploading model to {GCS_MODEL_PATH}...")
!gsutil -m cp -r {OUTPUT_DIR}/* {GCS_MODEL_PATH}

Uploading model to gs://boston311-chatbot-model/models/boston311-sqlcoder-tuned/...
Copying file://./sqlcoder_adapter/adapter_config.json [Content-Type=application/json]...
Copying file://./sqlcoder_adapter/adapter_model.safetensors [Content-Type=application/octet-stream]...
Copying file://./sqlcoder_adapter/README.md [Content-Type=text/markdown]...
Copying file://./sqlcoder_adapter/special_tokens_map.json [Content-Type=application/json]...
Copying file://./sqlcoder_adapter/tokenizer_config.json [Content-Type=application/json]...
Copying file://./sqlcoder_adapter/tokenizer.json [Content-Type=application/json]...
Copying file://./sqlcoder_adapter/tokenizer.model [Content-Type=application/octet-stream]...
\ [7/7 files][ 68.0 MiB/ 68.0 MiB] 100% Done                                    
Operation completed over 7 objects/68.0 MiB.                                     


In [38]:
print("Registering model in Vertex AI...")

# Note: We register the adapter. Serving this requires a custom container,
# but this step secures your model artifact in the official registry.

model_registry = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=GCS_MODEL_PATH,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/pytorch-gpu.2-1.py310:latest",
    description="Fine-tuned SQLCoder adapter for Boston 311 data",
    labels={"type": "lora-adapter", "source": "colab-training"}
)

print(f"Model Registered! Resource Name: {model_registry.resource_name}")

Registering model in Vertex AI...
Model Registered! Resource Name: projects/763245636613/locations/us-central1/models/1268263572895432704
